In [41]:

import cv2 as cv
import fitz
from PIL import Image
import numpy as np
import pandas as pd
from numpy import pi, sin, cos
import matplotlib.pyplot as plt

In [42]:
# Read all the images in the pdf created by camscanner
def getImagesFromPDF(file_name):
    img_lst = []
    doc = fitz.open(file_name)
    for i in range(len(doc)):
        for img in doc.get_page_images(i):
            xref = img[0]
            pix = fitz.Pixmap(doc, xref)
            if pix.n >= 5:       # this is CMYK (not GRAY or RGB) so convert to RGB first
                pix = fitz.Pixmap(fitz.csRGB, pix)

            mode = "RGBA" if pix.alpha else "RGB"
            img = np.array(Image.frombytes(mode, [pix.width, pix.height], pix.samples))
            if img.shape[0] > 270: # this is to skip the camscanner logo which is of shape (260, 260, 3)
                img_lst.append(("p%s-%s.png" % (i, xref), img))

            pix = None
    doc.close()

    return img_lst

In [43]:
# visualization functions
def drawLine(rho, theta, img):
    a = cos(theta)
    b = sin(theta)
    x0, y0 = a*rho, b*rho
    x1, y1 = int(x0 + 2000*(-b)), int(y0 + 2000*(a))
    x2, y2 = int(x0 - 2000*(-b)), int(y0 - 2000*(a))
    cv.line(img, (x1, y1), (x2, y2), (255,0,0), 2)

# computational functions
def getIntersectionPoint(line1, line2):
    rho1, theta1 = line1[0], line1[1]
    rho2, theta2 = line2[0], line2[1]
    x = int((rho1*sin(theta2)-rho2*sin(theta1))/(sin(theta2-theta1)))
    y = int((rho2*cos(theta1)-rho1*cos(theta2))/(sin(theta2-theta1)))
    return x, y

In [44]:
# constants
mcq_grid_orig_dim = (1109.6, 517.6)
table_size = tuple(map(int, mcq_grid_orig_dim))
mcq_grid_cell_marg = 5
border_width = 2

# Trims the answers table
def extractMCQTable(mcq_sheet, showExtraction=False):
    # fine tune constants
    canny_min_threshold = 20
    canny_max_threshold = 80
    canny_apertureSize = 3
    hough_votes_vert_density = 200  # makes the vote count dependant to the image dimensions
    hough_votes_hori_density = 500  # makes the vote count dependant to the image dimensions
    hough_theta_max_variation_deg = 2

    hough_votes_vert = int(hough_votes_vert_density/mcq_sheet.shape[0]*900)
    hough_votes_hori = int(hough_votes_hori_density/mcq_sheet.shape[1]*1200)
    hough_theta_max_variation = hough_theta_max_variation_deg/180*np.pi

    gray = cv.cvtColor(mcq_sheet, cv.COLOR_BGR2GRAY)

    # detect houghlines
    edges = cv.Canny(gray, canny_min_threshold, canny_max_threshold, canny_apertureSize)
    horizontal_lines = cv.HoughLines(edges, 1, np.pi/180, hough_votes_hori, min_theta=np.pi/2-hough_theta_max_variation, max_theta=np.pi/2+hough_theta_max_variation)
    vertical_lines = cv.HoughLines(edges, 1, np.pi/180, hough_votes_vert, min_theta=0, max_theta=+hough_theta_max_variation)
    
    left_most_line, right_most_line, top_most_line, bottom_most_line = None, None, None, None
    max_val, min_val = 0, np.inf
    for line in horizontal_lines:
        if line[0][0]<min_val:
            left_most_line = line[0]
            min_val = line[0][0]
        if line[0][0]>max_val:
            right_most_line = line[0]
            max_val = line[0][0]
    max_val, min_val = 0, np.inf
    for line in vertical_lines:
        if line[0][0]<min_val:
            top_most_line = line[0]
            min_val = line[0][0]
        if line[0][0]>max_val:
            bottom_most_line = line[0]
            max_val = line[0][0]

    detected_img = mcq_sheet.copy()
    for line in (left_most_line, right_most_line, top_most_line, bottom_most_line):
        drawLine(line[0], line[1], detected_img)

    # get the corner points
    top_left = getIntersectionPoint(top_most_line, left_most_line)
    top_right = getIntersectionPoint(top_most_line, right_most_line)
    bottom_left = getIntersectionPoint(bottom_most_line, left_most_line)
    bottom_right = getIntersectionPoint(bottom_most_line, right_most_line)
    for point in (top_left, top_right, bottom_left, bottom_right):
        cv.circle(detected_img, point, 5, (0,0,255), -1)

    # apply homography
    contract_val = border_width
    H = cv.getPerspectiveTransform(np.float32((top_left, top_right, bottom_right, bottom_left)), np.float32((
        (-contract_val, -contract_val),
        (-contract_val, +contract_val+mcq_grid_orig_dim[1]),
        (+contract_val+mcq_grid_orig_dim[0], +contract_val+mcq_grid_orig_dim[1]),
        (+contract_val+mcq_grid_orig_dim[0], -contract_val)
    )))
    mcq_table = cv.warpPerspective(mcq_sheet, H, table_size)

    if showExtraction:
        fig, ax = plt.subplots(1,3, figsize=(16,8))
        ax[0].imshow(edges, cmap="gray")
        ax[1].imshow(detected_img)
        ax[2].imshow(mcq_table)

        plt.show()

    return mcq_table

In [45]:
# Divides the regions and read the data

def getQuestionRegion(mcq_table, table_size, question_number):
    assert type(question_number) == int
    assert question_number <= 50
    question_region_padding = 5
    row = (question_number-1)%10
    col = int((question_number-1)/10)
    region_w = table_size[0]/5
    region_h = table_size[1]/10
    question_region = mcq_table[int(row*region_h+question_region_padding):int((row+1)*region_h-question_region_padding), int(col*region_w+question_region_padding):int((col+1)*region_w-question_region_padding)]

    return question_region

def getSelectedAnswer(mcq_table, table_size, question_number, showRegions=False):
    question_number_width, answers_width = 30, 178
    answers_region_padding = 4
    question_region = getQuestionRegion(mcq_table, table_size, question_number)
    question_width = question_region.shape[1]
    answers_region = question_region[:,int(answers_region_padding+question_width*question_number_width/(question_number_width + answers_width)):-answers_region_padding]
    answer_width = answers_region.shape[1]/5

    # select the answer which has been colored (darkest)
    min_intensity, selected_answer = np.inf, 0
    for i in range(5):
        intensity = answers_region[:,int(i*answer_width):int((i+1)*answer_width)].sum()
        if intensity < min_intensity:
            min_intensity = intensity
            selected_answer = i+1
    
    if showRegions:
        fig, ax = plt.subplots(1,5)
        for i in range(5):
            ax[i].imshow(answers_region[:,int(i*answer_width):int((i+1)*answer_width)])
        plt.show()
    
    return selected_answer

def exportData(data, filename):
    out_str = "name,selected_answers, correct, score\n"
    out_str += "\n".join(list(map(lambda x: ",".join(list(map(str, x))), data)))
    with open(filename, "w") as f:
        f.write(out_str)

In [73]:
def markMCQ(correct_answers_path, answer_sheet_pdf_path, names_list_path, output_path):
    correct_answers = pd.read_csv(correct_answers_path)
    img_lst = getImagesFromPDF(answer_sheet_pdf_path)
    names = None
    if names_list_path != "":
        names = pd.read_csv(names_list_path, header=None)
        assert len(names) == len(img_lst), "Answer script count in the pdf is not similar to the number of names"
        names = names.iloc()

    results = []

    for i, img in enumerate(img_lst):
        mcq_sheet = img[1]
        mcq_table = extractMCQTable(mcq_sheet)
        selected_answers = ""
        correct = ""
        score = 0
        for row in correct_answers.iloc():
            q_num = row["question"]
            correct_answer = row["answer"]
            selected_answer = getSelectedAnswer(mcq_table, table_size, int(q_num))
            selected_answers += str(selected_answer)
            if selected_answer == correct_answer:
                correct += "1"
                score += 1
            else:
                correct += "0"
                
        if names is not None:
            results.append((names[i][0], selected_answers, correct, score))
        else:
            results.append((selected_answers, correct, score))

    exportData(results, f"{output_path}/results.csv")
    
    return results
        

In [92]:
from tkinter.ttk import *
from tkinter import Tk, END, BOTTOM, LEFT, RIGHT
from tkinter import filedialog

class App(Tk):  
    def __init__(self):
        super().__init__()
        framePaddings = {'padx': 10, 'pady': 10}
        paddings = {'padx': 5, 'pady': 5}
        entry_font = {'font': ('Helvetica', 11)}
        fileSelFrame = Frame(self)
        fileSelFrame.pack(**framePaddings)

        correctAnsLabel = Label(fileSelFrame, text="Correct answers file")
        correctAnsEntry = Entry(fileSelFrame, width= 60, **entry_font)
        correctAnsSelBtn = Button(fileSelFrame, text="Select", command = lambda: self.browseFiles(correctAnsEntry, [1]))
        correctAnsLabel.grid(sticky="W", row=2,column=0, **paddings)
        correctAnsEntry.grid(row=2, column=1, **paddings)
        correctAnsSelBtn.grid(row=2, column=2, **paddings)

        ansSheetLabel = Label(fileSelFrame, text="Answer sheet pdf")
        ansSheetEntry = Entry(fileSelFrame, width= 60, **entry_font)
        ansSheetSelBtn = Button(fileSelFrame, text="Select", command = lambda: self.browseFiles(ansSheetEntry, [2]))
        ansSheetLabel.grid(sticky="W", row=4,column=0, **paddings)
        ansSheetSelBtn.grid(row=4, column=2, **paddings)
        ansSheetEntry.grid(row=4, column=1, **paddings)

        nameLstLabel = Label(fileSelFrame, text="Names list")
        nameLstEntry = Entry(fileSelFrame, width= 60, **entry_font)
        nameLstSelBtn = Button(fileSelFrame, text="Select", command = lambda: self.browseFiles(nameLstEntry, [1]))
        nameLstLabel.grid(sticky="W", row=6,column=0, **paddings)
        nameLstSelBtn.grid(row=6, column=2, **paddings)
        nameLstEntry.grid(row=6, column=1, **paddings)

        outputLabel = Label(fileSelFrame, text="Output location")
        outputEntry = Entry(fileSelFrame, width= 60, **entry_font)
        outputSelBtn = Button(fileSelFrame, text="Select", command = lambda: self.browseFiles(outputEntry, openFile=False))
        outputLabel.grid(sticky="W", row=8,column=0, **paddings)
        outputSelBtn.grid(row=8, column=2, **paddings)
        outputEntry.grid(row=8, column=1, **paddings)

        notificationFrame = Frame(self)
        notificationFrame.pack(**paddings)
        notificationLabel = Label(notificationFrame)
        notificationLabel.pack(**paddings)

        resultsFrame = Frame(self)
        resultsFrame.pack(**framePaddings)

        buttonFrame = Frame(self)
        buttonFrame.pack(side = BOTTOM, **framePaddings)
        generateBtn = Button(buttonFrame, text="Mark", command=self.mark)
        generateBtn.pack(side = LEFT)
        closeBtn = Button(buttonFrame, text="Close", command=self.destroy)
        closeBtn.pack(side = RIGHT)

        self.notificationLabel = notificationLabel
        self.correctAnsEntry = correctAnsEntry
        self.ansSheetEntry = ansSheetEntry
        self.nameLstEntry = nameLstEntry
        self.outputEntry = outputEntry
        self.resultsFrame = resultsFrame

        # delete these
        correctAnsEntry.insert(0, 'D:\\ACA\Projects\\05-mcq-marking\\research\\correct-answers.csv')
        ansSheetEntry.insert(0, 'D:\\ACA\Projects\\05-mcq-marking\\research\\grade-8-mcq-answer-sheets-2.pdf')
        outputEntry.insert(0, 'C:\\Users\\Avishka\\Desktop')

    def browseFiles(self, entry, fileTypeIndices=[], openFile=True):
        if openFile:
            fileTypesMap = {0:("Text files", "*.txt*"), 1:("CSV files", "*.csv*"), 2:("PDF files", "*.pdf*"), -1:("all files", "*.*")}
            filetypes = list(map(lambda index: fileTypesMap[index], fileTypeIndices))
            # filetypes.append(fileTypesMap[-1])
            text = filedialog.askopenfilename(title = "Select a File", filetypes = filetypes)
        else:
            text = filedialog.askdirectory(title = "Select a Folder")

        entry.delete(0,END)
        entry.insert(0,text)

    def mark(self):
        try:
            correct_answers_path, answer_sheet_pdf_path, names_list_path, output_path = self.correctAnsEntry.get(), self.ansSheetEntry.get(), self.nameLstEntry.get(), self.outputEntry.get()
            assert correct_answers_path != "", "Correct answers csv file must not be empty"
            assert answer_sheet_pdf_path != "", "Answer sheet pdf file must not be empty"
            assert output_path != "", "Output path must not be empty"
            print(correct_answers_path, answer_sheet_pdf_path, names_list_path, output_path)
            results = markMCQ(correct_answers_path, answer_sheet_pdf_path, names_list_path, output_path)
            self.show_results(results)
            self.notificationLabel.configure(text="Output saved!", background="lightgreen")
        except AssertionError as e:
            self.show_error(e)
        except PermissionError as e:
            if str(e)[-12:] == "results.csv'":
                self.show_error("Error opening the results file. Make sure that the old results file is not open")

    def show_error(self, message):
        self.notificationLabel.configure(text=message, background="pink")

    def show_results(self, results):
        # clear the frame
        for widget in self.resultsFrame.winfo_children():
            widget.destroy()

        containsNames = len(results[0]) == 4
        if containsNames: # if the names are included
            Label(self.resultsFrame, text="#").grid(row=0, column=0)
            Label(self.resultsFrame, text="Name").grid(row=0, column=1)
            Label(self.resultsFrame, text="Score").grid(row=0, column=2)
        else:
            Label(self.resultsFrame, text="#").grid(row=0, column=0)
            Label(self.resultsFrame, text="Score").grid(row=0, column=1)

        for i, result in enumerate(results):
            Label(self.resultsFrame, text=i).grid(row=i+1, column=0)
            if containsNames:
                Label(self.resultsFrame, text=results[i][0]).grid(row=i+1, column=1)
                Label(self.resultsFrame, text=results[i][-1]).grid(row=i+1, column=2)
            else:
                Label(self.resultsFrame, text=results[i][-1]).grid(row=i+1, column=1)


app = App()
app.mainloop()

D:\ACA\Projects\05-mcq-marking\research\correct-answers.csv D:\ACA\Projects\05-mcq-marking\research\grade-8-mcq-answer-sheets-2.pdf  C:\Users\Avishka\Desktop


In [79]:
names_list_path = ""
correct_answers_path, answer_sheet_pdf_path, output_path = "D:/ACA/Projects/05-mcq-marking/research/correct-answers.csv D:/ACA/Projects/05-mcq-marking/research/grade-8-mcq-answer-sheets-2.pdf  C:/Users/Avishka/Desktop".split()
markMCQ(correct_answers_path, answer_sheet_pdf_path, names_list_path, output_path)

PermissionError: [Errno 13] Permission denied: 'C:/Users/Avishka/Desktop/results.csv'